# Modelling with TF-IDF Vectorization


In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

In [5]:
yelp = pd.read_csv('yelp_cleaned_sentiscored.csv')
yelp.head()

,Comment,Rating,Clean_Comment,Tokenized_Comment,Lemmatized_Comment,Senti_Blob,Senti_norm
0,ich habe mir bewertungen zu restaurants auf ...,5,ich habe mir bewertungen zu restaurants auf ...,bewertungen restaurants menulist angesehen res...,ich haben sich bewertung zu restaurant auf...,1.000000,5.000000
1,das joseph ist ein ziemlich neues israelische...,4,das joseph ist ein ziemlich neues israelische...,joseph ziemlich neues israelisches restaurant ...,der joseph sein ein ziemlich neu israelisch ...,0.846429,4.692857
2,schalömchen scheiße haben wir uns heute kosche...,5,schalömchen scheiße haben wir uns heute kosche...,schalömchen scheiße heute koscher weggeknallt ...,schalömch scheiße haben wir sich heute kosch w...,0.283333,3.566667
3,essen war allenfalls ok muss ich nicht nochmal...,2,essen war allenfalls ok muss ich nicht nochmal...,essen allenfalls ok nochmal insgesamt erlebnis...,essen sein allenfalls ok mussen ich nicht noch...,0.133333,3.266667
4,waren auf empfehlung dort und wurden nicht en...,5,waren auf empfehlung dort und wurden nicht en...,empfehlung wurden enttäuscht hammer ambiente h...,sein auf empfehlung dort und werden nicht en...,0.750000,4.500000


In [6]:
# Q: https://melaniewalsh.github.io/Intro-Cultural-Analytics/05-Text-Analysis/03-TF-IDF-Scikit-Learn.html

#help(TfidfVectorizer)
# max_features: int, default=None --> If not None, build a vocabulary that only considers the top
# max_features ordered by term frequency across the corpus
# max/min_df: ignore terms that have a document frequency strictly higher/lower than given threshold
# setting lower and upper frequency thresholds as done in reference paper Luo, Xu
doc_frequency=0.8
min_absolute_frequency=5

tfidf = TfidfVectorizer(min_df=min_absolute_frequency, max_df=doc_frequency)
#tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(yelp['Lemmatized_Comment'])
print(X_tfidf)


  (0, 754)	0.4241610243114664
  (0, 4085)	0.10419735162670914
  (0, 485)	0.2912112864011511
  (0, 1800)	0.2063167666104816
  (0, 141)	0.20211466669651104
  (0, 3299)	0.347675355822079
  (0, 2573)	0.09819467679215284
  (0, 1343)	0.15684547169528526
  (0, 270)	0.38269074774378
  (0, 369)	0.12108218348116806
  (0, 4656)	0.2958710931984148
  (0, 6658)	0.09494880442320777
  (0, 855)	0.2724001551484482
  (0, 5119)	0.12829439166012327
  (0, 2595)	0.3070877667009214
  (0, 2896)	0.20798137667768696
  (1, 393)	0.12473831855268705
  (1, 2744)	0.047904446879624085
  (1, 4065)	0.05442354355065734
  (1, 2079)	0.11685699977158848
  (1, 936)	0.09692882882756876
  (1, 3957)	0.08633665216168444
  (1, 6052)	0.09849370149051788
  (1, 3488)	0.08856394946896615
  (1, 3958)	0.08747350446354939
  :	:
  (9704, 5049)	0.2736736445068207
  (9704, 4940)	0.2743118212468185
  (9704, 1805)	0.14725312067770818
  (9704, 1514)	0.23699397269705308
  (9704, 4085)	0.14681444203402041
  (9704, 6658)	0.1337832058642971
  (97

#### Split dataset

#### Model selection